In [2]:
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09c_changesig.ipynb"
_nb_title = "Changing signatures of exported model"

### no need to change any of this
_nb_safeloc = _nb_loc.replace('/', '%2F')
md("""
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name={1}&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F{2}&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F{2}">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/{0}">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
""".format(_nb_loc, _nb_title, _nb_safeloc))


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Changing signatures of exported model&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F09_deploying%2F09c_changesig.ipynb&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F09_deploying%2F09c_changesig.ipynb">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09c_changesig.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09c_changesig.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/09_deploying/09c_changesig.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


# Changing signatures of exported model

In this notebook, we start from an already trained and saved model (as in Chapter 7).
For convenience, we have put this model in a public bucket in gs://practical-ml-vision-book/flowers_5_trained

## Enable GPU and set up helper functions

This notebook and pretty much every other notebook in this repository
will run faster if you are using a GPU.
On Colab:
- Navigate to Edit→Notebook Settings
- Select GPU from the Hardware Accelerator drop-down

On Cloud AI Platform Notebooks:
- Navigate to https://console.cloud.google.com/ai-platform/notebooks
- Create an instance with a GPU or select your instance and add a GPU

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
print('TensorFlow version' + tf.version.VERSION)
print('Built with GPU support? ' + ('Yes!' if tf.test.is_built_with_cuda() else 'Noooo!'))
print('There are {} GPUs'.format(len(tf.config.experimental.list_physical_devices("GPU"))))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Exported model

We start from a trained and saved model from Chapter 7.
<pre>
  model.save(...)
</pre>

In [4]:
MODEL_LOCATION='gs://practical-ml-vision-book/flowers_5_trained'

In [5]:
!gsutil ls {MODEL_LOCATION}

gs://practical-ml-vision-book/flowers_5_trained/saved_model.pb
gs://practical-ml-vision-book/flowers_5_trained/variables/


In [6]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir {MODEL_LOCATION}

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:2
Method name is: tensorflow/serving/predict


## Passing through an input

Note that the signature doesn't tell us the input filename.
Let's add that.

In [8]:
import tensorflow as tf
import os, shutil
model = tf.keras.models.load_model(MODEL_LOCATION)

@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_flower_type(filenames):
    old_fn = model.signatures['serving_default']
    result = old_fn(filenames) # has flower_type_int etc.
    result['filename'] = filenames
    return result

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save('export/flowers_model',
          signatures={
              'serving_default': predict_flower_type
          })

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: export/flowers_model/assets


In [9]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir export/flowers_model

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:2
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:3
Method name is: tensorflow/serving/predict


In [10]:
import tensorflow as tf
serving_fn = tf.keras.models.load_model('export/flowers_model').signatures['serving_default']
filenames = [
    'gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg',
    'gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
    'gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg',
    'gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
    'gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg'
]
pred = serving_fn(tf.convert_to_tensor(filenames))
print(pred)

{'filename': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg',
       b'gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
       b'gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg',
       b'gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
       b'gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg'],
      dtype=object)>, 'probability': <tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.61915255, 0.9999844 , 0.995083  , 0.97518593, 0.954918  ],
      dtype=float32)>, 'flower_type_int': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 1, 0, 4, 4])>, 'flower_type_str': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'dandelion', b'dandelion', b'daisy', b'tulips', b'tulips'],
      dtype=object)>}


## Multiple signatures


In [12]:
import tensorflow as tf
import os, shutil
model = tf.keras.models.load_model(MODEL_LOCATION)
old_fn = model.signatures['serving_default']

@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def pass_through_input(filenames):
    result = old_fn(filenames) # has flower_type_int etc.
    result['filename'] = filenames
    return result

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save('export/flowers_model2',
          signatures={
              'serving_default': old_fn,
              'input_pass_through': pass_through_input
          })

INFO:tensorflow:Assets written to: export/flowers_model2/assets


In [13]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "input_pass_through"
SignatureDef key: "serving_default"


In [16]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2 --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall_1:0
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall_1:2
Method name is: tensorflow/serving/predict


In [17]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2 --signature_def input_pass_through

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: input_pass_through_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:2
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:3
Method name is: tensorflow/serving/predict


## Deploying multi-signature model as REST API

In [18]:
!./caip_deploy.sh --version multi --model_location ./export/flowers_model2

Deploying flowers:multi from ./export/flowers_model2
Using endpoint [https://ml.googleapis.com/]
The model named flowers already exists.
Using endpoint [https://ml.googleapis.com/]
Creating flowers:multi
Using endpoint [https://ml.googleapis.com/]


In [19]:
%%writefile request.json
{
    "instances": [
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [21]:
!gcloud ai-platform predict --model=flowers --version=multi --json-request=request.json

Using endpoint [https://ml.googleapis.com/]
FLOWER_TYPE_INT  FLOWER_TYPE_STR  PROBABILITY
1                dandelion        0.619152
1                dandelion        0.999984
0                daisy            0.995083
4                tulips           0.975186
4                tulips           0.954917


In [26]:
%%writefile request.json
{
    "signature_name": "input_pass_through",
    "instances": [
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [27]:
!gcloud ai-platform predict --model=flowers --version=multi --json-request=request.json

Using endpoint [https://ml.googleapis.com/]
FLOWER_TYPE_INT  FLOWER_TYPE_STR  PROBABILITY
1                dandelion        0.619152
1                dandelion        0.999984
0                daisy            0.995083
4                tulips           0.975186
4                tulips           0.954917


that's a bug ... filed a bug report; hope it's fixed by the time you are reading the book.

In [29]:
from oauth2client.client import GoogleCredentials
import requests
import json

PROJECT = 'ai-analytics-solutions' # CHANGE
MODEL_NAME = 'flowers'
MODEL_VERSION = 'multi'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
    "signature_name": "input_pass_through",
    "instances": [
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content.decode('utf-8'))

{"predictions": [{"filename": "gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg", "flower_type_int": 1, "flower_type_str": "dandelion", "probability": 0.6191519498825073}, {"filename": "gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg", "flower_type_int": 1, "flower_type_str": "dandelion", "probability": 0.9999843835830688}, {"filename": "gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg", "flower_type_int": 0, "flower_type_str": "daisy", "probability": 0.9950828552246094}, {"filename": "gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg", "flower_type_int": 4, "flower_type_str": "tulips", "probability": 0.9751859307289124}, {"filename": "gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg", "flower_type_int": 4, "flower_type_str": "tulips", "probability": 0.954916775226593}]}


## License
Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.